# Illustration of AEROBLADE (Fig. 1)

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
from aeroblade.data import ImageFolder
from aeroblade.distances import distance_from_config
from aeroblade.image import compute_reconstructions
from aeroblade.paper import configure_mpl, plot_tensor, set_figsize

output_dir = Path("output/00/idea")
output_dir.mkdir(parents=True, exist_ok=True)

configure_mpl()
set_figsize(format="double", factor=0.26)

dist = distance_from_config("lpips_vgg_0", spatial=True, batch_size=1, num_workers=1)
filename = "000000664.png"

# real
ds_real = ImageFolder(Path(f"data/raw/real/{filename}"))
ds_real_rec = ImageFolder(
    compute_reconstructions(
        ds_real,
        repo_id="stabilityai/stable-diffusion-2-base",
        output_root=Path("data/reconstructions"),
    )[0]
)
real_dist = -dist.compute(ds_real, ds_real_rec)[0]["lpips_vgg_0"]

plot_tensor(ds_real[0][0])
plt.savefig(output_dir / "real.pdf")
plot_tensor(ds_real_rec[0][0])
plt.savefig(output_dir / "real_rec.pdf")
plot_tensor(
    ds_real[0][0], real_dist[0][0], vmin=0.05, vmax=0.45, show_cbar=False, alpha=1.0
)
plt.savefig(output_dir / "real_lpips.pdf")

# SD2.1
ds_sd21 = ImageFolder(
    Path(
        f"data/raw/generated/stabilityai-stable-diffusion-2-1-base-ViT-H-14-laion2b_s32b_b79k/{filename}"
    )
)
ds_sd21_rec = ImageFolder(
    compute_reconstructions(
        ds_sd21,
        repo_id="stabilityai/stable-diffusion-2-base",
        output_root=Path("data/reconstructions"),
    )[0]
)
sd21_dist = -dist.compute(ds_sd21, ds_sd21_rec)[0]["lpips_vgg_0"]

plot_tensor(ds_sd21[0][0])
plt.savefig(output_dir / "fake.pdf")
plot_tensor(ds_sd21_rec[0][0])
plt.savefig(output_dir / "fake_rec.pdf")
plot_tensor(ds_sd21[0][0], sd21_dist[0][0], vmin=0.05, vmax=0.45, alpha=1.0)
plt.savefig(output_dir / "fake_lpips.pdf")
plt.close()

# Perturbation Examples (Fig. 12)

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import requests
from aeroblade.image import compute_reconstructions
from aeroblade.paper import configure_mpl, plot_tensor, set_figsize
from aeroblade.transforms import transform_from_config
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image

output_dir = Path("output/00/perturbation_examples")
output_dir.mkdir(parents=True, exist_ok=True)

image = Image.open(
    requests.get(
        "https://raw.githubusercontent.com/scikit-image/scikit-image/v0.21.0/skimage/data/astronaut.png",
        stream=True,
    ).raw
)

config_label_pairs = [
    ("jpeg_90", "JPEG ($q=90$)"),
    ("jpeg_80", "JPEG ($q=80$)"),
    ("jpeg_70", "JPEG ($q=70$)"),
    ("jpeg_60", "JPEG ($q=60$)"),
    ("jpeg_50", "JPEG ($q=50$)"),
    ("crop_0.9", "Crop ($f=0.9$)"),
    ("crop_0.8", "Crop ($f=0.8$)"),
    ("crop_0.7", "Crop ($f=0.7$)"),
    ("crop_0.6", "Crop ($f=0.6$)"),
    ("crop_0.5", "Crop ($f=0.5$)"),
    ("blur_1.0", "Blur ($\sigma = 1.0$)"),
    ("blur_2.0", "Blur ($\sigma = 2.0$)"),
    ("blur_3.0", "Blur ($\sigma = 3.0$)"),
    ("blur_4.0", "Blur ($\sigma = 4.0$)"),
    ("blur_5.0", "Blur ($\sigma = 5.0$)"),
    ("noise_0.05", "Noise ($\sigma = 0.05$)"),
    ("noise_0.1", "Noise ($\sigma = 0.1$)"),
    ("noise_0.15", "Noise ($\sigma = 0.15$)"),
    ("noise_0.2", "Noise ($\sigma = 0.2$)"),
    ("noise_0.25", "Noise ($\sigma = 0.25$)"),
]

configure_mpl()
set_figsize("double")
plt.rcParams.update(
    {"figure.dpi": 600, "axes.grid": False, "axes.linewidth": 0.5, "axes.labelsize": 5}
)
fig = plt.figure()
grid = ImageGrid(
    fig,
    111,
    nrows_ncols=(4, 5),
    axes_pad=0.125,
    direction="row",
    share_all=True,
    label_mode="all",
)

for ax, (config, label) in zip(grid, config_label_pairs):
    tf = transform_from_config(config)
    perturbed = tf(image)
    ax.imshow(np.array(perturbed))
    ax.set_xlabel(label)
grid[0].set_xticks([])
grid[0].set_yticks([])
plt.savefig(output_dir / "all.pdf")
plt.close()